In [3]:
import logging
import typing as T

from csv import DictReader
from io import StringIO

In [ ]:
INPUT_PATH = '~/Desktop/software_analysis_projects/entity_resolution/data/external/csv_example_messy_input.csv'
df = pd.read_csv(INPUT_PATH)


In [6]:
class EmailAddress(str):
    def __new__(cls, email: str):
        if '@' not in email:
            raise ValueError('invalid email')
        return super().__new__(cls, email)


class IpV4Address(str):
    def __new__(cls, address: str):
        if not len(address.split('.')) == 4:
            raise ValueError('ip address must have 4 octets: ' + address)
        elif not all(0 <= int(octet) <= 255 for octet in address.split('.')):
            raise ValueError('octet in ip address out-of-range: ' + address)
        
        return super().__new__(cls, address)
        
class Person(T.NamedTuple):
    id: int
    first_name: str
    last_name: str
    email: EmailAddress
    gender: str
    ip_address: IpV4Address
    
    @classmethod
    def from_row(cls, row: dict):
        return cls(**{
            key: type_(row[key]) for key, type_ in cls._field_types.items()
        })
    

def validate_csv(reader: DictReader) -> bool:
    for row in reader:
        try:
            Person.from_row(row)
        except Exception as e:
            logging.error('type: {} msg: {}'.format(type(e), e))
            return False
    return True

In [8]:
MOCK_DATA = '''
id,first_name,last_name,email,gender,ip_address
1,Tobiah,Giacovazzo,tgiacovazzo0@zdnet.com,Male,19.149.44.115
2,Shir,Januszewicz,sjanuszewicz1@acquirethisname.com,Female,153.89.169.230
3,Gaspar,Guitt,gguitt2@usa.gov,Male,174.171.101.164
4,Dun,Feakins,dfeakins3@indiatimes.com,Male,110.176.209.154
5,Tymon,Filewood,tfilewood4@pbs.org,Male,40.90.194.55
6,Ludvig,Urpeth,lurpeth5@technorati.com,Male,4.123.109.132
7,Tawnya,Genthner,tgenthner6@posterous.com,Female,238.58.35.55
8,Lukas,Gershom,lgershom7@ibm.com,Male,241.38.25.208
9,Doroteya,Mottley,dmottley8@drupal.org,Female,145.28.139.222

'''.strip()


mock_data_csv_reader = DictReader(StringIO(MOCK_DATA))

In [11]:
#print(validate_csv(mock_data_csv_reader))

print(validate_csv(DictReader('foo'))) 

ERROR:root:type: <class 'KeyError'> msg: 'id'


False


In [14]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column

def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d

In [15]:
INPUT_PATH = '~/Desktop/software_analysis_projects/entity_resolution/data/external/csv_example_messy_input.csv'


In [16]:
print(readData(INPUT_PATH))

FileNotFoundError: [Errno 2] No such file or directory: '~/Desktop/software_analysis_projects/entity_resolution/data/external/csv_example_messy_input.csv'